In [1]:
# Change drive locatio
cd drive/MyDrive/github/HandGestureRecognition/HandLandmarkModel/

/content/drive/MyDrive/github/HandGestureRecognition


In [2]:
import pickle
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

In [3]:
## load the normalized dataset
df = pd.read_csv('asl_alphabet_train/labels.csv')

x_norm = [f'x_{i}' for i in range(21)]
y_norm = [f'y_{i}' for i in range(21)]

df_x = df[x_norm]
df_y = df[y_norm]

df_x = df_x.subtract(df_x.min(axis=1), axis=0)
df_x = df_x.div(df_x.max(axis=1), axis=0)

df_y = df_y.subtract(df_y.min(axis=1), axis=0)
df_y = df_y.div(df_y.max(axis=1), axis=0)

df[x_norm] = df_x
df[y_norm] = df_y

df = df.sample(frac=1).reset_index(drop=True)       
x = df.iloc[:,4:].values
y = df['label'].values.reshape(-1,1)

In [4]:
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, C = 0.1, class_weight='balanced', verbose=1, fit_intercept=True)

# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# evaluate the model and collect the scores
n_scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)

In [5]:
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

Mean Accuracy: 0.894 (0.004)


In [6]:
# fit the model on the whole dataset
model.fit(x, y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   50.3s finished


LogisticRegression(C=0.1, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='multinomial', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=1, warm_start=False)

In [7]:
# Check model score on the whole dataset
model.score(x,y)

0.8962736014470017

In [8]:
# save the model to disk
filename = 'trained_models/LogReg_landmarks_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [9]:
# load the model from disk
filename = 'trained_models/LogReg_landmarks_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

# verify if the loaded model score is same as the saved model
result = loaded_model.score(x, y)
print(result)

0.8962736014470017
